In [2]:
import pandas as pd
import numpy as np
import deepchem as dc

import mlflow
import sys
sys.path.insert(0, '..')
from pathlib import Path
import src

In [3]:
# artifacts such as saved model weights, pickle files, etc
model_path = 'model_fit/content/model_fit/'
metadata_path = str(Path('..','metadata.txt'))
artifacts = {'model_files': model_path,
            'metadata': metadata_path}


In [9]:
# Serve as an MLflow wrapper for model
class ModelWrapper(mlflow.pyfunc.PythonModel):
    
    # Load in model and all required artifacts
    # context object is provided by mlflow
    # it contains all artifacts
    def load_context(self, context):
        import deepchem as dc
        import pickle
        from src import model as model_fn
        
        # load model
        model = dc.models.GraphConvModel(12, model_dir=context.artifacts['model_files'], )
        model.restore()
        
        self.model = model
    
    # function which takes in pandas df and returns predicted labels 
    def predict(self, context, model_input):
        from src import model as model_fn
        data = model_input['data'].item()
        return self.model.predict(data)

In [10]:
# mlflow complains if directory already exists, so remove it before saving model to mlflow directory
!rm -rf ./mlflow/model
src_path = Path('../src')
mlflowpath= Path('mlflow','model')

mlflow.pyfunc.save_model(path=str(mlflowpath), python_model=ModelWrapper(), 
                         artifacts=artifacts, conda_env='dc_env.yml', 
                         code_path=['../src/'])

In [11]:
# load model from mlflow directory
loaded_model = mlflow.pyfunc.load_model(str(mlflowpath))

In [12]:
tox21_tasks_2, tox21_datasets_2, transformers_2 = dc.molnet.load_tox21(featurizer='GraphConv')
train_dataset_2, valid_dataset_2, test_dataset_2 = tox21_datasets_2

In [13]:
df = pd.DataFrame({'data': valid_dataset_2}, index=[0])
df['data'].item()

<DiskDataset X.shape: (783,), y.shape: (783, 12), w.shape: (783, 12), ids: ['N#C[C@@H]1CC(F)(F)CN1C(=O)CNC1CC2CCC(C1)N2c1ncccn1'
 'CN(C)C(=O)NC1(c2ccccc2)CCN(CCC[C@@]2(c3ccc(Cl)c(Cl)c3)CCCN(C(=O)c3ccccc3)C2)CC1'
 'CSc1nnc(C(C)(C)C)c(=O)n1N' ...
 'O=C(O[C@H]1CN2CCC1CC2)N1CCc2ccccc2[C@@H]1c1ccccc1'
 'C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@H]3C(=C)C[C@@]21CC'
 'NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3c(c2)CCO3)C1'], task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>

In [14]:
loaded_model.predict(df)

/home/ryan/miniconda3/envs/dc_env_src4/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:538: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


AttributeError: 'Tensor' object has no attribute 'numpy'